In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
import re
from underthesea import word_tokenize
warnings.filterwarnings('ignore')

In [2]:
df_org = pd.read_csv('2k_data_Sep.csv')
df = df_org.copy()
df.shape

(2000, 8)

In [9]:
df.head()

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
0,4416.0,b) \( \left(\frac{-4}{15}-\frac{18}{19}\right)...,a) \( \frac{2}{3}+\frac{-1}{3}+\frac{7}{15} \)...,NEGATIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/9a7aa76...,https://s3.icankid.io/uploads/question/41bf56d...
1,4419.0,Đề số \( 1 . \)\n1. Tính\nb) \( \sqrt{25} \)\n...,b) \( \sqrt{25} \)\nc) \( \sqrt{-36} \)\na) \(...,NEGATIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/0398020...,https://s3.icankid.io/uploads/question/5a47b89...
2,4420.0,1. Tính: \( \sqrt{\frac{25}{4}}+\left(\sqrt{\f...,Đề số \( 3 . \)\n1. Tính: \( \sqrt{\frac{25}{4...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/a81b075...,https://s3.icankid.io/uploads/question/2d3358a...
3,4423.0,Câu 49: Cho hàm số \( y=f(x) \) có đạo hàm là ...,Câu 49. Cho hàm số \( y=f(x) \) có đạo hàm \( ...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/a981260...,https://s3.icankid.io/uploads/question/aaa4976...
4,4425.0,b. Giải phương trình: \( 2 \sin ^{2} x+\sqrt{3...,1) \( \sin ^{2} 2 x-(2+\sqrt{3}) \sin 2 x+\sqr...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/caa519f...,https://s3.icankid.io/uploads/question/7ea6f5f...


In [5]:
dup_row_lookup = df['Lookup Text'].duplicated().sum()
dup_row_lookup

43

In [6]:
dup_row_question = df['Question Text'].duplicated().sum()
dup_row_question

168

In [7]:
dup_row_question / dup_row_lookup

3.9069767441860463

In [8]:
df[['Lookup Text', 'Question Text']].duplicated().sum()

38

In [13]:
check_dup = df[['Lookup Text', 'Question Text']][df[['Lookup Text', 'Question Text']].duplicated()]
check_dup.shape

(38, 2)

In [17]:
df_dup = df.iloc[check_dup.index, :]
df_dup

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
263,5169.0,Viết khai triển theo công thức nhị thức Niu - ...,Viết khai triển theo công thức nhị thức Niu-tơ...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/fb220e2...,https://s3.stag.icankids.com.vn/uploads/questi...
274,5196.0,Chứng minh rằng phương trình \( x^{5}-3 x^{4}+...,Chứng minh răng phương trình \( x^{5}-3 x^{4}+...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/f9d1889...,https://s3.icankid.io/uploads/question/00b6366...
284,5218.0,Cho tam giác \( A B C \) vuông cân tại \( A \)...,Câu 126: Cho tam giác \( A B C \) vuông cân tạ...,POSITIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/0a73cef...,https://s3.icankid.io/uploads/question/a9127c4...
307,5284.0,3. Viết mỗi tập hợp sau bằng cách liệt kê các ...,3. Viết mỗi tập họp sau bằng cách liệt kê các ...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/ec52e86...,https://s3.stag.icankids.com.vn/uploads/questi...
333,5345.0,"Trong mặt phẳng tọa độ \( O x y \), trên các t...","6. Trong mặt phẳng tọa độ \( O x y \), trên cá...",POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/26522c9...,https://s3.stag.icankids.com.vn/uploads/questi...
379,5458.0,Bim \( m \) dé haim sớ dóng biến trén \( R \)....,"5A. Cho ba đường thẳng:\n\[\nd_{1}: y=x-4, d_{...",NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/ae48916...,https://s3.icankid.io/uploads/question/012fd63...
380,5460.0,Bim \( m \) dé haim sớ dóng biến trén \( R \)....,"5A. Cho ba đường thẳng:\n\[\nd_{1}: y=x-4, d_{...",NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/ae48916...,https://s3.icankid.io/uploads/question/012fd63...
381,5461.0,Bim \( m \) dé haim sớ dóng biến trén \( R \)....,"5A. Cho ba đường thẳng:\n\[\nd_{1}: y=x-4, d_{...",NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/ae48916...,https://s3.icankid.io/uploads/question/012fd63...
382,5464.0,Bim \( m \) dé haim sớ dóng biến trén \( R \)....,"5A. Cho ba đường thẳng:\n\[\nd_{1}: y=x-4, d_{...",NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/ae48916...,https://s3.icankid.io/uploads/question/012fd63...
524,5877.0,1.6. Tuổi thọ trung bình\n\begin{tabular}{|c|c...,\begin{tabular}{|c|c|c|c|c|c|}\n\hline \multic...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/9105c33...,https://s3.icankid.io/uploads/question/6742c98...


In [27]:
ss = df.iloc[307]['Lookup Text']

In [28]:
df.loc[df['Lookup Text'] == ss]

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
183,4939.0,3. Viết mỗi tập hợp sau bằng cách liệt kê các ...,3. Viết mỗi tập họp sau bằng cách liệt kê các ...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/01dd3b7...,https://s3.stag.icankids.com.vn/uploads/questi...
307,5284.0,3. Viết mỗi tập hợp sau bằng cách liệt kê các ...,3. Viết mỗi tập họp sau bằng cách liệt kê các ...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/ec52e86...,https://s3.stag.icankids.com.vn/uploads/questi...


In [81]:
dup_index = df_dup.index
df_dup_lookup_question = pd.DataFrame(columns = df.columns)
diff = []
for idx in dup_index:
    dup_lookup_text = df.iloc[idx]['Lookup Text']
    df_temp = df.loc[df['Lookup Text'] == dup_lookup_text]
    if len(df_temp['Feedback Type'].unique()) != 1 or len(df_temp['Human Feedback'].unique()) != 1:
        diff.append(idx)
    df_dup_lookup_question = pd.concat([df_dup_lookup_question, df_temp], axis = 0)

print(len(diff))

0


In [84]:
df_dup_lookup_question

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
262,5168.0,Viết khai triển theo công thức nhị thức Niu - ...,Viết khai triển theo công thức nhị thức Niu-tơ...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/fb220e2...,https://s3.stag.icankids.com.vn/uploads/questi...
263,5169.0,Viết khai triển theo công thức nhị thức Niu - ...,Viết khai triển theo công thức nhị thức Niu-tơ...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/fb220e2...,https://s3.stag.icankids.com.vn/uploads/questi...
273,5195.0,Chứng minh rằng phương trình \( x^{5}-3 x^{4}+...,Chứng minh răng phương trình \( x^{5}-3 x^{4}+...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/f9d1889...,https://s3.icankid.io/uploads/question/00b6366...
274,5196.0,Chứng minh rằng phương trình \( x^{5}-3 x^{4}+...,Chứng minh răng phương trình \( x^{5}-3 x^{4}+...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/f9d1889...,https://s3.icankid.io/uploads/question/00b6366...
279,5209.0,Cho tam giác \( A B C \) vuông cân tại \( A \)...,Câu 126: Cho tam giác \( A B C \) vuông cân tạ...,POSITIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/a84ede2...,https://s3.icankid.io/uploads/question/a9127c4...
...,...,...,...,...,...,...,...,...
1763,9269.0,Câu 9. Với \( n \) là số nguyên dương bất kì \...,Câu 9. Với \( n \) là số nguyên dương bất kì \...,NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/15d8a15...,https://s3.icankid.io/uploads/question/a7957b9...
1780,9328.0,8.Tính hợp lý (nếu có thê)\na) \( 2 \frac{3}{4...,Bài 1. Tính nhanh:\na) \( \frac{-3}{7}+\frac{1...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/da83f42...,https://s3.icankid.io/uploads/question/b978f76...
1783,9336.0,8.Tính hợp lý (nếu có thê)\na) \( 2 \frac{3}{4...,Bài 1. Tính nhanh:\na) \( \frac{-3}{7}+\frac{1...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/7cab87d...,https://s3.icankid.io/uploads/question/b978f76...
1924,9780.0,Cho tam giác \( A B C \) nhọn \( (A B>A C) \) ...,Cho tam giác \( A B C \) nhọn \( (A B>A C) \) ...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/0e1f4e1...,https://s3.icankid.io/uploads/question/945ae57...


In [43]:
dup_lookup = df[df['Lookup Text'].duplicated()]
dup_lookup.shape

(43, 8)

In [44]:
dup_lookup

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
263,5169.0,Viết khai triển theo công thức nhị thức Niu - ...,Viết khai triển theo công thức nhị thức Niu-tơ...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/fb220e2...,https://s3.stag.icankids.com.vn/uploads/questi...
274,5196.0,Chứng minh rằng phương trình \( x^{5}-3 x^{4}+...,Chứng minh răng phương trình \( x^{5}-3 x^{4}+...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/f9d1889...,https://s3.icankid.io/uploads/question/00b6366...
284,5218.0,Cho tam giác \( A B C \) vuông cân tại \( A \)...,Câu 126: Cho tam giác \( A B C \) vuông cân tạ...,POSITIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/0a73cef...,https://s3.icankid.io/uploads/question/a9127c4...
307,5284.0,3. Viết mỗi tập hợp sau bằng cách liệt kê các ...,3. Viết mỗi tập họp sau bằng cách liệt kê các ...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/ec52e86...,https://s3.stag.icankids.com.vn/uploads/questi...
333,5345.0,"Trong mặt phẳng tọa độ \( O x y \), trên các t...","6. Trong mặt phẳng tọa độ \( O x y \), trên cá...",POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/26522c9...,https://s3.stag.icankids.com.vn/uploads/questi...
379,5458.0,Bim \( m \) dé haim sớ dóng biến trén \( R \)....,"5A. Cho ba đường thẳng:\n\[\nd_{1}: y=x-4, d_{...",NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/ae48916...,https://s3.icankid.io/uploads/question/012fd63...
380,5460.0,Bim \( m \) dé haim sớ dóng biến trén \( R \)....,"5A. Cho ba đường thẳng:\n\[\nd_{1}: y=x-4, d_{...",NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/ae48916...,https://s3.icankid.io/uploads/question/012fd63...
381,5461.0,Bim \( m \) dé haim sớ dóng biến trén \( R \)....,"5A. Cho ba đường thẳng:\n\[\nd_{1}: y=x-4, d_{...",NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/ae48916...,https://s3.icankid.io/uploads/question/012fd63...
382,5464.0,Bim \( m \) dé haim sớ dóng biến trén \( R \)....,"5A. Cho ba đường thẳng:\n\[\nd_{1}: y=x-4, d_{...",NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/ae48916...,https://s3.icankid.io/uploads/question/012fd63...
524,5877.0,1.6. Tuổi thọ trung bình\n\begin{tabular}{|c|c...,\begin{tabular}{|c|c|c|c|c|c|}\n\hline \multic...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/9105c33...,https://s3.icankid.io/uploads/question/6742c98...


In [47]:
lst_a = [x for x in dup_lookup.index if x not in df_dup.index]
len(lst_a)

5

In [50]:
df_a = df.iloc[lst_a, :]
df_a

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
878,6783.0,\( (2 x-1)^{2}-2(2 x-1)(2 x+1)+(2 x+1)^{2} \),\( (2 x-1)^{2}-2(2 x-1)(2 x+1)+(2 x+1)^{2} \),POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/b6eb446...,https://s3.icankid.io/uploads/question/cbb28d1...
1113,7453.0,a) \( \left(x^{2}-1\right)\left(x^{2}+2 x\righ...,a) \( \left(x^{2}-1\right)\left(x^{2}+2 x\righ...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/7b2011e...,https://s3.icankid.io/uploads/question/fd5c2d7...
1265,7879.0,3) \( 2\left(\frac{3}{4}-5 x\right)=\frac{4}{5...,3) \( 2\left(\frac{3}{4}-5 x\right)=\frac{4}{5...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/87aefdf...,https://s3.icankid.io/uploads/question/0e38490...
1606,8805.0,\( \left(\frac{-7}{13}\right) \cdot \frac{5}{1...,\( \left(\frac{-7}{13}\right) \cdot \frac{5}{1...,POSITIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/0cf4771...,https://s3.icankid.io/uploads/question/3fcefe6...
1839,9489.0,"Bai 1\n\[\n2, \frac{-30}{17}: \frac{-15}{-34}=...","Bai 1\n\[\n2, \frac{-30}{17}: \frac{-15}{-34}=...",POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/753bc56...,https://s3.icankid.io/uploads/question/5548250...


In [52]:
rr = df.iloc[878]['Lookup Text']
rr

'\\( (2 x-1)^{2}-2(2 x-1)(2 x+1)+(2 x+1)^{2} \\)'

In [53]:
df.loc[df['Lookup Text'] == rr]

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
868,6758.0,\( (2 x-1)^{2}-2(2 x-1)(2 x+1)+(2 x+1)^{2} \),1 Bài 1 : Tính\n\[\n1 /(2 x-1)^{2}+(2 x+1)^{2}...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/cbb28d1...,https://s3.icankid.io/uploads/question/fcc62ef...
878,6783.0,\( (2 x-1)^{2}-2(2 x-1)(2 x+1)+(2 x+1)^{2} \),\( (2 x-1)^{2}-2(2 x-1)(2 x+1)+(2 x+1)^{2} \),POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/b6eb446...,https://s3.icankid.io/uploads/question/cbb28d1...


In [55]:
df_dup_lookup_only = pd.DataFrame(columns = df.columns)
for idx_ in lst_a:
    rr = df.iloc[idx_]['Lookup Text']
    df_temp = df.loc[df['Lookup Text'] == rr]
    df_dup_lookup_only = pd.concat([df_dup_lookup_only, df_temp], axis = 0)

In [56]:
df_dup_lookup_only

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
868,6758.0,\( (2 x-1)^{2}-2(2 x-1)(2 x+1)+(2 x+1)^{2} \),1 Bài 1 : Tính\n\[\n1 /(2 x-1)^{2}+(2 x+1)^{2}...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/cbb28d1...,https://s3.icankid.io/uploads/question/fcc62ef...
878,6783.0,\( (2 x-1)^{2}-2(2 x-1)(2 x+1)+(2 x+1)^{2} \),\( (2 x-1)^{2}-2(2 x-1)(2 x+1)+(2 x+1)^{2} \),POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/b6eb446...,https://s3.icankid.io/uploads/question/cbb28d1...
1111,7450.0,a) \( \left(x^{2}-1\right)\left(x^{2}+2 x\righ...,A. \( (x-1)^{2}(x+1)\left(x^{2}+x+1\right) \)\...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/fd5c2d7...,https://s3.icankid.io/uploads/question/9497f83...
1113,7453.0,a) \( \left(x^{2}-1\right)\left(x^{2}+2 x\righ...,a) \( \left(x^{2}-1\right)\left(x^{2}+2 x\righ...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/7b2011e...,https://s3.icankid.io/uploads/question/fd5c2d7...
1261,7872.0,3) \( 2\left(\frac{3}{4}-5 x\right)=\frac{4}{5...,"Bài 4: Tìm x, biết:\n1. \( 4 x +\frac{1}{3}=\f...",NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/0e38490...,https://s3.icankid.io/uploads/question/e7acefb...
1265,7879.0,3) \( 2\left(\frac{3}{4}-5 x\right)=\frac{4}{5...,3) \( 2\left(\frac{3}{4}-5 x\right)=\frac{4}{5...,POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/87aefdf...,https://s3.icankid.io/uploads/question/0e38490...
854,6717.0,\( \left(\frac{-7}{13}\right) \cdot \frac{5}{1...,4. Tính:\na) \( \frac{3}{7} \cdot\left(-\frac{...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/3fcefe6...,https://s3.icankid.io/uploads/question/ed6be8e...
1606,8805.0,\( \left(\frac{-7}{13}\right) \cdot \frac{5}{1...,\( \left(\frac{-7}{13}\right) \cdot \frac{5}{1...,POSITIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/0cf4771...,https://s3.icankid.io/uploads/question/3fcefe6...
1648,8939.0,"Bai 1\n\[\n2, \frac{-30}{17}: \frac{-15}{-34}=...",Bài 5: Thực hiện các phép tính sau:\n\( A =\fr...,NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/753bc56...,https://s3.icankid.io/uploads/question/60ebbbd...
1839,9489.0,"Bai 1\n\[\n2, \frac{-30}{17}: \frac{-15}{-34}=...","Bai 1\n\[\n2, \frac{-30}{17}: \frac{-15}{-34}=...",POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/753bc56...,https://s3.icankid.io/uploads/question/5548250...


In [86]:
df.iloc[1648]['Lookup Text']

'Bai 1\n\\[\n2, \\frac{-30}{17}: \\frac{-15}{-34}=\n\\]'

In [88]:
df.iloc[1839]['Question Text']

'Bai 1\n\\[\n2, \\frac{-30}{17}: \\frac{-15}{-34}=\n\\]'

In [76]:
lst_a

[878, 1113, 1265, 1606, 1839]

In [57]:
dup_question = df[df['Question Text'].duplicated()]
dup_question.shape

(168, 8)

In [58]:
dup_question

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
47,4537.0,"Câu 1 ( 2,0 điểm).\n1) Giaii phương trinh: \( ...",Trung tâm toán Yến Ly\nĐÉ THI THǗ TUYÉN SINH V...,NEGATIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/704f232...,https://s3.icankid.io/uploads/question/8fc6220...
111,4721.0,"a) \( 4=3,7+4,3-x \mid \)\nb) \( B \)\nd) \( D...",6/ Tìm x biết:\na) \( 2 x(x-5)-x(3+2 x)=26 \)\...,NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/3fb21e7...,https://s3.icankid.io/uploads/question/948af29...
167,4889.0,"Câu 49. Có 2 hộp đụ̂ng bi: hộp I có 2 bi đỏ, 1...",D. \( \frac{4}{7} \)\nCâu 50. Có 2 hộp đựng bi...,POSITIVE,NOT_MATCH,MATCH,https://s3.icankid.io/uploads/question/025530c...,https://s3.icankid.io/uploads/question/d309309...
183,4939.0,3. Viết mỗi tập hợp sau bằng cách liệt kê các ...,3. Viết mỗi tập họp sau bằng cách liệt kê các ...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/01dd3b7...,https://s3.stag.icankids.com.vn/uploads/questi...
216,5037.0,"2. Cho tam giác \( ABC \) vuông tại \( A \), đ...","Cho tam giác \( ABC \) vuông tại \( A \), đườn...",POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/b58a964...,https://s3.icankid.io/uploads/question/462fa27...
...,...,...,...,...,...,...,...,...
1939,9813.0,b) \( \left[3^{4}-\left(8^{2}+14\right): 13\ri...,b) \( \left[3^{4}-\left(8^{2}+14\right): 13\ri...,POSITIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/8c01869...,https://s3.icankid.io/uploads/question/330af57...
1945,9822.0,"b) \( 2 \frac{3}{8} \cdot(-0,4) \)","2. Tính:\na) \( 5,75 \cdot \frac{-8}{9} \);\nb...",NEGATIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/0f6fbcb...,https://s3.icankid.io/uploads/question/ea79685...
1949,9830.0,\( 5^{9}: 5^{7} \)\n\( 2^{11}: 2^{8} \)\n\( 3^...,a) Viết kết quả mỗi phép tính sau dưới dạng mộ...,NEGATIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/4d2ccda...,https://s3.icankid.io/uploads/question/8767a68...
1974,9909.0,( iul 14: hang sin \( 2^{2}+\sin 4^{\circ}+\si...,Câu 30. Tổng \( \sin ^{2} 2^{\circ}+\sin ^{2} ...,NEGATIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/390b096...,https://s3.icankid.io/uploads/question/764b21f...


In [63]:
qq = df.iloc[47]['Question Text']
qq

'Trung tâm toán Yến Ly\nĐÉ THI THǗ TUYÉN SINH VÀO LÓP 10\nTHPT\nPHÒNG GIÁO DỤC VẢ ĐẢO TẠO NĂM HỌC 2022 - 2023ĐÈ THI MÔN: TOÁN\n(Thời gian 120 phút không kể thời gian giao đề)\nCâu 1 (2,0 điểm) Giài phương trình và hệ phương trình sau:\na) \\( x(2 x-3)=5 \\)\nb) \\( \\left\\{\\begin{array}{l}2(x-1)+y=3 \\\\ x-3(y+2)=-14\\end{array}\\right. \\)\nCâu 2 (2,0 điểm)\na) Cho hàm số bậc nhất \\( y=(m-2) x+m^{2}-2 m+3 \\) có đồ thị là (d). Tìm giá trị của \\( m \\) để đồ thị hàm sồ đi qua giao điểm của 2 đường thẳng sau:\n\\( 2 x+y=5 \\) và \\( y=-1+4 x \\)\nb) Rút gọn biểu thức \\( A=\\left(\\frac{1}{\\sqrt{x}-2}-\\frac{5 \\sqrt{x}-4}{x-2 \\sqrt{x}}\\right):\\left(\\frac{2+\\sqrt{x}}{\\sqrt{x}}-\\frac{\\sqrt{x}}{\\sqrt{x}-2}\\right)+1 \\) với \\( x>0 \\) và \\( x \\neq 4 \\).\nCâu 3 (2,0 điểm)\na) Một đội công nhân theo kế hoạch phải sản xuất 120 sàn phẩm, nhưng đến khi thực hiện công việc không những 2 công nhân được điều đi làm việc khác mà tội còn được giao thêm 30 sản phẩm nữa. Vì vậy, đề 

In [64]:
df.loc[df['Question Text'] == qq]

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
14,4447.0,Cho biếu thức \( A=\left(\frac{x^{2}}{x^{3}-4 ...,Trung tâm toán Yến Ly\nĐÉ THI THǗ TUYÉN SINH V...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/77dcae7...,https://s3.icankid.io/uploads/question/8fc6220...
47,4537.0,"Câu 1 ( 2,0 điểm).\n1) Giaii phương trinh: \( ...",Trung tâm toán Yến Ly\nĐÉ THI THǗ TUYÉN SINH V...,NEGATIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/704f232...,https://s3.icankid.io/uploads/question/8fc6220...


In [66]:
lst_b = [x for x in dup_question.index if x not in df_dup.index]
len(lst_b)

130

In [85]:
lst_b1 = [x for x in lst_b if x not in df_dup_lookup_question.index]
len(lst_b1)

128

In [67]:
df_b = df.iloc[lst_b, :]
df_b.shape

(130, 8)

In [74]:
df_dup_question_only = pd.DataFrame(columns = df.columns)
for idx_ in lst_b:
    qq = df.iloc[idx_]['Question Text']
    df_temp = df.loc[df['Question Text'] == qq]
    df_dup_question_only = pd.concat([df_dup_question_only, df_temp], axis = 0)

In [75]:
df_dup_question_only

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
14,4447.0,Cho biếu thức \( A=\left(\frac{x^{2}}{x^{3}-4 ...,Trung tâm toán Yến Ly\nĐÉ THI THǗ TUYÉN SINH V...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/77dcae7...,https://s3.icankid.io/uploads/question/8fc6220...
47,4537.0,"Câu 1 ( 2,0 điểm).\n1) Giaii phương trinh: \( ...",Trung tâm toán Yến Ly\nĐÉ THI THǗ TUYÉN SINH V...,NEGATIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/704f232...,https://s3.icankid.io/uploads/question/8fc6220...
12,4445.0,Cho biéu thức: \( B=\left(\frac{x-4}{x \cdot(x...,6/ Tìm x biết:\na) \( 2 x(x-5)-x(3+2 x)=26 \)\...,NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/4e20218...,https://s3.icankid.io/uploads/question/948af29...
111,4721.0,"a) \( 4=3,7+4,3-x \mid \)\nb) \( B \)\nd) \( D...",6/ Tìm x biết:\na) \( 2 x(x-5)-x(3+2 x)=26 \)\...,NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/3fb21e7...,https://s3.icankid.io/uploads/question/948af29...
1782,9331.0,Bài tûp cho HS ngày 20/9\nBai 1. Tinh:\n4. \( ...,6/ Tìm x biết:\na) \( 2 x(x-5)-x(3+2 x)=26 \)\...,NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/3ccc04b...,https://s3.icankid.io/uploads/question/948af29...
...,...,...,...,...,...,...,...,...
1949,9830.0,\( 5^{9}: 5^{7} \)\n\( 2^{11}: 2^{8} \)\n\( 3^...,a) Viết kết quả mỗi phép tính sau dưới dạng mộ...,NEGATIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/4d2ccda...,https://s3.icankid.io/uploads/question/8767a68...
806,6600.0,Tổng \( \sin ^{2} 2^{\circ}+\sin ^{2} 4^{\circ...,Câu 30. Tổng \( \sin ^{2} 2^{\circ}+\sin ^{2} ...,NEGATIVE,SKIP,NOT MATCH,https://s3.icankid.io/uploads/question/94e2de8...,https://s3.icankid.io/uploads/question/764b21f...
1974,9909.0,( iul 14: hang sin \( 2^{2}+\sin 4^{\circ}+\si...,Câu 30. Tổng \( \sin ^{2} 2^{\circ}+\sin ^{2} ...,NEGATIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/390b096...,https://s3.icankid.io/uploads/question/764b21f...
1640,8916.0,"8. Vào tháng 6, giá niêm yết một chiếc ti vi 4...","8. Vào tháng 6 , giá niêm yết một chiếc ti vi ...",POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/4a51201...,https://s3.icankid.io/uploads/question/9363c3c...


In [80]:
df_dup_question_only[df_dup_question_only['Lookup Text'].duplicated()]

,Feedback ID,Lookup Text,Question Text,Feedback Type,Feedback Result,Human Feedback,Lookup Image,Question Image
307,5284.0,3. Viết mỗi tập hợp sau bằng cách liệt kê các ...,3. Viết mỗi tập họp sau bằng cách liệt kê các ...,POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/ec52e86...,https://s3.stag.icankids.com.vn/uploads/questi...
198,4973.0,"1.3. Bằng cách liệt kê các phần tử, hãy viết c...","1.3. Bằng cách liệt kê các phần tử, hãy viết c...",POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/8b1247c...,https://s3.stag.icankids.com.vn/uploads/questi...
247,5134.0,"1.3. Bằng cách liệt kê các phần tử, hãy viết c...","1.3. Bằng cách liệt kê các phần tử, hãy viết c...",POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/b8c2245...,https://s3.stag.icankids.com.vn/uploads/questi...
250,5138.0,"1.3. Bà̀ng cách liệt kê các phần tử, hãy viết ...","1.3. Bằng cách liệt kê các phần tử, hãy viết c...",POSITIVE,SKIP,MATCH,https://s3.icankid.io/uploads/question/bd75a4d...,https://s3.stag.icankids.com.vn/uploads/questi...
458,5700.0,"1.3. Bằng cách liệt kê các phần tử, hãy viết c...","1.3. Bằng cách liệt kê các phần tử, hãy viết c...",POSITIVE,MATCH,MATCH,https://s3.icankid.io/uploads/question/fd10fc2...,https://s3.stag.icankids.com.vn/uploads/questi...
...,...,...,...,...,...,...,...,...
348,5380.0,BA/ TẠp 6. Dọ đọan văn tử Khi ánh binh minh đă...,(3 điểm)\nĐọc đoạn thơ sau và thục hiện các yê...,NEGATIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/341957c...,https://s3.icankid.io/uploads/question/cce515d...
349,5385.0,TINHHUONG\nM. lả học sinh giôi toàn diện. Dến ...,(3 điểm)\nĐọc đoạn thơ sau và thục hiện các yê...,NEGATIVE,MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/04385b9...,https://s3.icankid.io/uploads/question/cce515d...
603,6099.0,1.13. Bảng sau cho biết các điểm đông đặc và đ...,(3 điểm)\nĐọc đoạn thơ sau và thục hiện các yê...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/a1bf35a...,https://s3.icankid.io/uploads/question/cce515d...
1452,8373.0,âu 41. [Mức độ 3] Một cuộc khảo sát thói quen ...,(3 điểm)\nĐọc đoạn thơ sau và thục hiện các yê...,NEGATIVE,NOT_MATCH,NOT MATCH,https://s3.icankid.io/uploads/question/a44a204...,https://s3.icankid.io/uploads/question/cce515d...


In [4]:
df = df.drop_duplicates().reset_index(drop = True)
df.shape

(2000, 8)

In [ ]:
def clean_function(text):  
    #lowercase
    text = text.lower()
    
#     #xoá ký tự đặc biệt, ký tự không phải alphanumeric 
#     text = re.sub("[^a-z0-9àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễòóọỏõôồốộổỗơờớợởỡìíịỉĩùúụủũưừứựửữỳýỵỷỹđ]",' ', text)
    
#     #xoá leading zero
#     text = [e.lstrip('0') for e in text.split(' ')]
#     text = (' ').join(text)
    
    #replace extra spaces with single space
    text = re.sub(' +',' ',text)
    
    #xoá khoảng trắng ở đầu và cuối mỗi câu
    text = text.strip()
    
    #word tokenzie
    text = word_tokenize(text, format='text')
    return text

In [ ]:
df['clean_lookup'] = df['Lookup Text'].map(lambda x: clean_function(x))
df['clean_question'] = df['Question Text'].map(lambda x: clean_function(x))

In [ ]:
df['char_count_lookup'] = df['clean_lookup'].str.len()
df['word_count_lookup'] = df['clean_lookup'].str.split().str.len()
df['average_word_length_lookup'] = df['char_count_lookup'] / df['word_count_lookup']

df['char_count_question'] = df['clean_question'].str.len()
df['word_count_question'] = df['clean_question'].str.split().str.len()
df['average_word_length_question'] = df['char_count_question'] / df['word_count_question']

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
human_fb = pd.DataFrame(df['Human Feedback'].value_counts())

sb.set_theme()
plt.figure(figsize=(10, 6))
sb.barplot(data = human_fb, x = human_fb.index, y = 'Human Feedback')
plt.grid(True);

In [ ]:
mask1 = df['Feedback Type'] == 'POSITIVE'
mask2 = df['Human Feedback'] == 'NOT MATCH'

mask3 = df['Feedback Type'] == 'NEGATIVE'
mask4 = df['Human Feedback'] == 'MATCH'

se_neg = df[(mask1 & mask2) | (mask3 & mask4)]
se_neg.shape

In [ ]:
cur_acc = df.shape

In [ ]:
plt.figure(figsize=(20,6))
plt.subplot(121)
sb.distplot(df_pos.words_lookup)
plt.subplot(122)
sb.distplot(df_pos.words_question)
plt.suptitle('Distribution of number of words in question of user and answer of search engine')
plt.show()

In [ ]:
mask9 = df['Feedback Type'] == 'NEGATIVE'
mask10 = df['Human Feedback'] == 'MATCH'

df[mask9 & mask10]

In [ ]:
mask11 = df['Feedback Type'] == 'POSITIVE'
mask12 = df['Human Feedback'] == 'NOT MATCH'

df[mask11 & mask12]

In [ ]:
(156 + 177) / 2000 * 100